In [ ]:
import numpy as np
import os
from skimage.transform import hough_line, hough_line_peaks
from skimage.feature import canny
from skimage.draw import line
from skimage import data
import matplotlib.pyplot as plt
from matplotlib import cm
import cv2
from skimage import filters
import scipy.ndimage as ndi

def non_max_suppression(heatmap, window_size=3):
    """
    Applies non-maximum suppression to a heatmap
    
    Arguments:
    heatmap -- The input heatmap
    window_size -- The size of the suppression window (default 3)
    
    Returns:
    The coordinates of the starting point
    """
    # Find local maxima
    local_max = ndi.maximum_filter(heatmap, size=window_size, mode='constant')
    maxima = (heatmap == local_max)
    maxima &= (heatmap > 0)
    
    # Find coordinates of maxima
    coordinates = np.column_stack(np.nonzero(maxima))
    
    # Select the starting point
    starting_point = coordinates[heatmap[coordinates[:, 0], coordinates[:, 1]].argmax()]
    
    return starting_point

def non_local_means(heatmap, h=1, sigma=1):
    """
    Apply Non-local Means filtering to a heatmap
    
    Parameters:
        heatmap (numpy array): The input heatmap to be filtered
        h (float): The filter strength (smaller h = stronger filter)
        sigma (float): The standard deviation of the Gaussian kernel
    
    Returns:
        numpy array: The filtered heatmap
    """
    filtered_heatmap = cv2.fastNlMeansDenoising(heatmap, h=h, templateWindowSize=3, searchWindowSize=15)
    return filtered_heatmap

def apply_erosion(heatmap, kernel_size=6):
    """
    Apply erosion to a heatmap
    
    Parameters:
        heatmap (numpy array): The input heatmap to be eroded
        kernel_size (int): The size of the structuring element used for erosion
    
    Returns:
        numpy array: The eroded heatmap
    """
    # Define a structuring element for erosion
    kernel = np.ones((kernel_size, kernel_size), np.uint8)
    
    # Apply erosion to the heatmap
    eroded_heatmap = cv2.erode(heatmap, kernel)
    
    return eroded_heatmap


def apply_median_filter(heatmap, kernel_size=9):
    """
    Apply median filtering to a heatmap
    
    Parameters:
        heatmap (numpy array): The input heatmap to be filtered
        kernel_size (int): The size of the sliding window used for median filtering
    
    Returns:
        numpy array: The filtered heatmap
    """
    filtered_heatmap = cv2.medianBlur(heatmap, kernel_size)
    return filtered_heatmap

path = os.chdir('C:\\Users\\z004b1tz\\Desktop\\Master Thesis Project\\Multi-class UNET\\Dataset\\output\\erosion')

for file in os.listdir(path):
    filename = os.fsdecode(file)
    image = cv2.imread(filename)
    starting_point = non_max_suppression(image)
    print(starting_point)
    # Group the indices by the row coordinate
    grouped_rows = {}
    for i, row in enumerate(rows):
        if row not in grouped_rows:
            grouped_rows[row] = []
        grouped_rows[row].append(cols[i])

    # Calculate the average x and y coordinates for each group of green pixels
    coordinates = []
    for row, cols_group in grouped_rows.items():
        x = int(np.mean(cols_group))
        y = int(row)
        coordinates.append((x, y))
    print(coordinates)
    plt.imsave('C:\\Users\\z004b1tz\\Desktop\\Master Thesis Project\\Multi-class UNET\\Dataset\\output\\pred\\'+filename, image)

In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

path = os.chdir('C:\\Users\\z004b1tz\\Desktop\\Master Thesis Project\\Multi-class UNET\\Dataset\\output\\erosion')

# Define the colormap
colormap = np.array([[0, 0, 0], [0, 0, 0], [255, 0, 0]])

for file in os.listdir(path):
    filename = os.fsdecode(file)
    image = cv2.imread(filename)
    image = np.array(image[:,:,0])
    # Convert the predictions to an image using the colormap
    image = colormap[image.astype(int)].astype(np.uint8)

    # Save the image
    plt.imsave('C:\\Users\\z004b1tz\\Desktop\\Master Thesis Project\\Multi-class UNET\\Dataset\\output\\pred\\'+filename, image)

In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

def apply_erosion(heatmap, kernel_size=15):
    """
    Apply erosion to a heatmap
    
    Parameters:
        heatmap (numpy array): The input heatmap to be eroded
        kernel_size (int): The size of the structuring element used for erosion
    
    Returns:
        numpy array: The eroded heatmap
    """
    # Define a structuring element for erosion
    kernel = np.ones((kernel_size, kernel_size), np.uint8)
    
    # Apply erosion to the heatmap
    eroded_heatmap = cv2.erode(heatmap, kernel)
    
    return eroded_heatmap

path = os.chdir('C:\\Users\\z004b1tz\\Desktop\\Master Thesis Project\\Multi-class UNET\\Dataset\\output\\predictions')

for file in os.listdir(path):
    filename = os.fsdecode(file)
    image = cv2.imread(filename,0)
    retval, binary = cv2.threshold(image, 225, 255, cv2.THRESH_BINARY)
    result = apply_erosion(binary)
    # Save the image
    plt.imsave('C:\\Users\\z004b1tz\\Desktop\\Master Thesis Project\\Multi-class UNET\\Dataset\\output\\erosion\\'+filename, result)

In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd

def apply_erosion(heatmap, kernel_size=15):
    # Define a structuring element for erosion
    kernel = np.ones((kernel_size, kernel_size), np.uint8)
    
    # Apply erosion to the heatmap
    eroded_heatmap = cv2.erode(heatmap, kernel)
    
    return eroded_heatmap

path = os.chdir('C:\\Users\\z004b1tz\\Desktop\\Master Thesis Project\\Multi-class UNET\\Dataset\\output\\preds')
gt_path = 'C:\\Users\\z004b1tz\\Desktop\\Master Thesis Project\\ground'

detected = {'filename':[], 'detected_point':[]}
true, false = 0,0
for file in os.listdir(path):
    filename = os.fsdecode(file)
    ground = filename.replace('.PNG', '.raw.csv')
    image = cv2.imread(filename,0)
    image = apply_erosion(image)
    retval, binary = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY)
    contours, hierarchy = cv2.findContours(binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cells = pd.read_csv(gt_path+'\\'+ground)
    if len(cells) == len(contours):
        true += 1
    else:
        false += 1
        break
    contours_image = cv2.drawContours(binary, contours, -1, (255,0,0), 3)
    detected['filename'].append(filename)
#     plt.imsave('C:\\Users\\z004b1tz\\Desktop\\Master Thesis Project\\Multi-class UNET\\Dataset\\output\\New folder\\'+filename,binary)
    for contour in contours:
        areas.append(cv2.contourArea(contour))
        x,y,w,h = cv2.boundingRect(contour)
#         print(x,y,w,h)
        cv2.rectangle(contours_image,(x,y),(x+w,y+h),(0,0,255),2)
        cv2.circle(contours_image, (x+w//2, y+h//2), 5, (255, 255, 255), -1)
        x_detected = x+w//2
        y_detected = y+h//2
        point = list((x_detected, y_detected))
        detected['detected_point'].append(point)
    print(filename)
    plt.imshow(contours_image)
    plt.show()

In [ ]:
areas
areas_filtered = [x for x in areas if x > 10]
print(len(areas_filtered))

In [ ]:
np.array(areas_filtered)
plt.hist(areas_filtered,bins = 300)
plt.title('Contour areas distribution')
plt.xlabel('Area')
plt.ylabel('Frequency')
plt.show()